In [66]:
import geopandas as gpd
import pandas as pd
import json
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
import os

In [67]:
import sys,os

### Wczytanie danych

* Dane o gestości zaludnienia na 1km^2 w woj. mazowieckim w latach 2008-2018, GUS: 
https://bdl.stat.gov.pl/BDL/dane/podgrup/wymiary
* Dane o położeniu gmin w Polsce: http://www.gugik.gov.pl/pzgik/dane-bez-oplat/dane-z-panstwowego-rejestru-granic-i-powierzchni-jednostek-podzialow-terytorialnych-kraju-prg (jednostki_administracyjne.zip --> pliki .shp)

Pliki znajdują się w folderze ../data/jednostki_administracyjne/ i ../data/dane_gus/

In [68]:
shapefile = os.path.realpath('jednostki_administracyjne/Gminy.shp')
#Read shapefile using Geopandas
gdf = gpd.read_file(shapefile,  encoding='utf-8')[['JPT_KOD_JE', 'JPT_NAZWA_', 'geometry']]
gdf.columns = ['TERYT', 'gmina', 'geometry']

In [70]:
gdf.head(10)

,TERYT,gmina,geometry
0,2602042,Nagłowice,"POLYGON ((20.13718983500007 50.59346325100006,..."
1,2602063,Sędziszów,"POLYGON ((20.03891401300007 50.50016492200007,..."
2,2602012,Imielno,"POLYGON ((20.45329288600004 50.52786629900004,..."
3,2604012,Bieliny,"POLYGON ((20.96474800000004 50.77738094900008,..."
4,2604112,Mniów,"POLYGON ((20.50759639400007 50.95792621800007,..."
5,2605033,Końskie,"POLYGON ((20.47150822000003 51.08160355100006,..."
6,2605083,Stąporków,"POLYGON ((20.56541267800003 51.04709407100006,..."
7,2606053,Ożarów,"POLYGON ((21.68909441700004 50.77308044900008,..."
8,2609032,Klimontów,"POLYGON ((21.40563099600007 50.59093404300006,..."
9,2609082,Wilczyce,"POLYGON ((21.69682773200003 50.70775861500005,..."


In [69]:
datafile = os.path.realpath('dane_gus/gestosc_zalud.csv') 
#Read csv file using pandas
df = pd.read_csv(datafile, sep=';')
df.columns=['TERYT', 'Nazwa', '2008','2009','2010','2011','2012','20013','2014','2015','2016','2017','2018', '']
df = df.drop(columns='')
df.TERYT =df.TERYT.apply(lambda x: str(x))


yr = 2016
df_yr = df[['TERYT', 'Nazwa', str(yr)]]
df_yr.columns = ['TERYT', 'Nazwa', 'gestosc_zaludnienia']

#Merge dataframes gdf and df_2008
merged = gdf.set_index('TERYT').merge(df_yr.set_index('TERYT'), on='TERYT')

In [71]:
merged.head(10)

,gmina,geometry,Nazwa,gestosc_zaludnienia
TERYT,,,,
1433062,Miedzna,"POLYGON ((22.00727682200005 52.43433143600004,...",Miedzna (2),34.0
1433032,Korytnica,"POLYGON ((21.82672519300007 52.34647400700004,...",Korytnica (2),35.0
1403011,Garwolin,"POLYGON ((21.63427702700005 51.86821642700005,...",Garwolin (1),785.0
1403032,Borowie,"POLYGON ((21.72966408800005 51.91019250400007,...",Borowie (2),65.0
1403143,Żelechów,"POLYGON ((21.77670720100002 51.75953359100004,...",Żelechów (3),96.0
1410012,Huszlew,"POLYGON ((22.90218751100002 52.06352558800006,...",Huszlew (2),24.0
1410023,Łosice,"POLYGON ((22.77328869300004 52.15228020500007,...",Łosice (3),90.0
1406042,Goszczyn,"POLYGON ((20.78951497600008 51.69902413800003,...",Goszczyn (2),52.0
1406022,Błędów,"POLYGON ((20.69138910900006 51.72872945000006,...",Błędów (2),57.0


In [ ]:
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column
#Define function that returns json_data for year selected by user.
    
def json_data(selectedYear):
    ## import pandas as pd
    datafile = os.path.realpath('dane_gus/gestosc_zalud.csv') 
    #Read csv file using pandas
    df = pd.read_csv(datafile, sep=';')
    df.columns=['TERYT', 'Nazwa', '2008','2009','2010','2011','2012','20013','2014','2015','2016','2017','2018', '']
    df = df.drop(columns='')
    df.TERYT =df.TERYT.apply(lambda x: str(x))

    
    yr = selectedYear
    df_yr = df[['TERYT', 'Nazwa', str(yr)]]
    df_yr.columns = ['TERYT', 'Nazwa', 'gestosc_zaludnienia']
    
    #Merge dataframes gdf and df_2008
    merged = gdf.set_index('TERYT').merge(df_yr.set_index('TERYT'), on='TERYT')
    
    #merged = gdf.merge(df_yr, left_on = 'country_code', right_on =     'code', how = 'left')
    merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data
#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data(2016))
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 4000, nan_color = '#d9d9d9')
#Define custom tick labels for color bar.
#tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#Add hover tool
hover = HoverTool(tooltips = [ ('gmina',' @gmina'),('gestosc zaludnienia', '@gestosc_zaludnienia')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal') #, major_label_overrides = tick_labels)

#Create figure object.
p = figure(title = 'Gęstość zaludnienia w woj. Mazowieckim w 2016r.', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'gestosc_zaludnienia', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify layout
p.add_layout(color_bar, 'below')
# Define the callback function: update_plot
def update_plot(attr, old, new):
    print('wybrany rok: ', slider.value)
    yr = slider.value
    new_data = json_data(yr)
    geosource.geojson = new_data
    p.title.text = 'Gęstość zaludnienia w woj. Mazowieckim w %d r.' %yr
    
# Make a slider object: slider 
slider = Slider(title = 'Rok',start = 2008, end = 2018, step = 1, value = 2016)
slider.on_change('value', update_plot)
# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)
#Display plot inline in Jupyter notebook
output_notebook()
#Display plot
show(layout)


In [40]:
gdf.head(10)

,TERYT,gmina,geometry
0,2602042,Nagłowice,"POLYGON ((20.13718983500007 50.59346325100006,..."
1,2602063,Sędziszów,"POLYGON ((20.03891401300007 50.50016492200007,..."
2,2602012,Imielno,"POLYGON ((20.45329288600004 50.52786629900004,..."
3,2604012,Bieliny,"POLYGON ((20.96474800000004 50.77738094900008,..."
4,2604112,Mniów,"POLYGON ((20.50759639400007 50.95792621800007,..."
5,2605033,Końskie,"POLYGON ((20.47150822000003 51.08160355100006,..."
6,2605083,Stąporków,"POLYGON ((20.56541267800003 51.04709407100006,..."
7,2606053,Ożarów,"POLYGON ((21.68909441700004 50.77308044900008,..."
8,2609032,Klimontów,"POLYGON ((21.40563099600007 50.59093404300006,..."
9,2609082,Wilczyce,"POLYGON ((21.69682773200003 50.70775861500005,..."


In [42]:
merged.head(10)

,gmina,geometry,Nazwa,gestosc_zaludnienia
TERYT,,,,
1433062,Miedzna,"POLYGON ((22.00727682200005 52.43433143600004,...",Miedzna (2),34.0
1433032,Korytnica,"POLYGON ((21.82672519300007 52.34647400700004,...",Korytnica (2),35.0
1403011,Garwolin,"POLYGON ((21.63427702700005 51.86821642700005,...",Garwolin (1),785.0
1403032,Borowie,"POLYGON ((21.72966408800005 51.91019250400007,...",Borowie (2),65.0
1403143,Żelechów,"POLYGON ((21.77670720100002 51.75953359100004,...",Żelechów (3),96.0
1410012,Huszlew,"POLYGON ((22.90218751100002 52.06352558800006,...",Huszlew (2),24.0
1410023,Łosice,"POLYGON ((22.77328869300004 52.15228020500007,...",Łosice (3),90.0
1406042,Goszczyn,"POLYGON ((20.78951497600008 51.69902413800003,...",Goszczyn (2),52.0
1406022,Błędów,"POLYGON ((20.69138910900006 51.72872945000006,...",Błędów (2),57.0
